In [44]:
!which python

/Users/apabook/Desktop/ds-deployment/.venv/bin/python


In [45]:
import pandas as pd
import requests
import json
import pprint


# Testing HL Api for PUSH transfer

---

## Consts

In [46]:
data_space_provider = "http://127.0.0.1:1234"
data_space_consumer = "http://127.0.0.1:1235"
context_broker = "http://fiware-orion:1026"
streaming_processor = "http://streaming-testing-service:1237"

catalog_id = ""
dataservice_id = ""
agreement_id = ""
agreement_pid = ""

## Setup Catalog, Dataservice and agreements

In [47]:
payload = {
    "foaf:homepage": "My catalog in Dataspace provider",
    "dct:title": "My catalog in Dataspace provider"
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/catalogs"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
catalog_id = response_as_json["@id"]

pprint.pprint(response.json())

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@id': '8853e3b8-bf22-4d53-8dd5-dae467bfea28',
 '@type': 'dcat:Catalog',
 'dcat:dataset': [],
 'dcat:keyword': '',
 'dcat:service': [],
 'dcat:theme': '',
 'dct:conformsTo': None,
 'dct:creator': None,
 'dct:description': [],
 'dct:identifier': '8853e3b8-bf22-4d53-8dd5-dae467bfea28',
 'dct:issued': '2024-11-26T12:11:33.873277',
 'dct:modified': None,
 'dct:title': 'My catalog in Dataspace provider',
 'dspace:extraFields': None,
 'dspace:participantId': None,
 'foaf:homepage': 'My catalog in Dataspace provider',
 'odrl:hasPolicy': None}


In [48]:
suscription_payload = {
    "description": "Air Quality Unit Subscription",
    "subject": {
        "entities":[
            {"idPattern":".*","type":"AirQualityUnit"}],
        "condition": {
            "attrs":["CO"]
        }
    },
    "notification": {
        "http": {
            "url":"$data_url",
        },
    },
    "expires":"2040-01-01T14:00:00.00Z",
    "throttling":5
}
payload = {
    "dcat:endpointURL": context_broker + "/v2/subscriptions",
    "dcat:endpointDescription": json.dumps(suscription_payload),
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/catalogs/" + catalog_id + "/data-services"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
dataservice_id = response_as_json["@id"]

pprint.pprint(response.json())

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@id': 'dc88d015-ec56-4231-95a8-caf1910ad0ce',
 '@type': 'dcat:DataService',
 'dcat:endpointDescription': '{"description": "Air Quality Unit Subscription", '
                             '"subject": {"entities": [{"idPattern": ".*", '
                             '"type": "AirQualityUnit"}], "condition": '
                             '{"attrs": ["CO"]}}, "notification": {"http": '
                             '{"url": "$data_url"}}, "expires": '
                             '"2040-01-01T14:00:00.00Z", "throttling": 5}',
 'dcat:endpointURL': 'http://fiware-orion:1026/v2/subscriptions',
 'dcat:keyword': '',
 'dcat:theme': '',
 'dct:conformsTo': None,
 'dct:creator': None,
 'dct:description': [],
 'dct:identifier': 'dc88d015-ec56-4231-95a8-caf1910ad0ce',
 'dct:issued': '2024-11-26T12:11:33.945992',
 'dct:modified': None,
 'dct:title': None,
 'dspace:extraFields': None,
 'odrl:hasPolicy': None}


In [49]:
payload = {
    "dataServiceId": dataservice_id
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/agreements"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
agreement_id = response_as_json["agreement_id"]
agreement_pid = "urn:uuid:" + agreement_id

pprint.pprint(response.json())

{'agreement_id': '837fcbf0-bb0f-4816-8761-059d989678e2',
 'data_service_id': 'dc88d015-ec56-4231-95a8-caf1910ad0ce',
 'identity': None,
 'identity_token': None}


## Setup transfer

In [50]:
url = data_space_consumer + "/api/v1/setup-transfer"

payload = ""
headers = {}
response = requests.request("POST", url, headers=headers, data=payload)
callbackAddress = response.json()["callbackAddress"]
callbackId = response.json()["callbackId"]
consumerPid = response.json()["consumerPid"]

pprint.pprint(response.json())

{'callbackAddress': 'http://ds-consumer:1235/2e4cc854-0e1a-4261-930c-997ab84f59fc',
 'callbackId': '2e4cc854-0e1a-4261-930c-997ab84f59fc',
 'consumerPid': 'urn:uuid:a220cdad-9e0b-4248-a59c-73733b4fbd1e'}


## Request transfer

In [51]:
url = data_space_consumer + "/api/v1/request-transfer"

payload = json.dumps({
    "agreementId": agreement_pid,
    "format": "fiware+push",
    "dataAddress": {
        "@type": "dspace:DataAddress",
        "dspace:endpoint": streaming_processor + "/data-client",
        "dspace:endpointType": "HTTP",
        "dspace:endpointProperties": []
    },
    "callbackAddress": callbackAddress,
    "callbackId": callbackId,
    "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())


{'consumerPid': 'urn:uuid:a220cdad-9e0b-4248-a59c-73733b4fbd1e',
 'error': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
           '@type': 'dspace:TransferError',
           'dspace:code': 'TRANSFER_ERROR_CODE',
           'dspace:consumerPid': '123',
           'dspace:providerPid': '123',
           'dspace:reason': ['Internal server error',
                             'Not checked error.',
                             'error sending request for url '
                             '(http://fiware-orion:1026/v2/subscriptions)']}}


#### For seeing how the push service is working, please check the logs in the streaming-testing-service

## Suspend transfer

In [52]:
url = data_space_consumer + "/api/v1/suspend-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

## Test data access has been suspended


#### For seeing how the push service is no longer receiving data, please check the logs in the streaming-testing-service

## Restart transfer

In [33]:
url = data_space_consumer + "/api/v1/restart-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:5d2ec721-4cb2-4735-9141-59acfbfded12',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:5d2ec721-4cb2-4735-9141-59acfbfded12',
                     'dspace:providerPid': 'urn:uuid:3712845c-27c1-4532-916d-70ee1098ee67',
                     'dspace:state': 'dspace:STARTED'}}


## Test data is accessible again

#### For seeing how the push service is working, please check the logs in the streaming-testing-service

## Complete transfer

In [34]:
url = data_space_consumer + "/api/v1/complete-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:5d2ec721-4cb2-4735-9141-59acfbfded12',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:5d2ec721-4cb2-4735-9141-59acfbfded12',
                     'dspace:providerPid': 'urn:uuid:3712845c-27c1-4532-916d-70ee1098ee67',
                     'dspace:state': 'dspace:COMPLETED'}}


## Data is not accessible

#### For seeing how the push service is no longer receiving data, please check the logs in the streaming-testing-service

## Protocol state cannot change

In [35]:
url = data_space_consumer + "/api/v1/restart-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:5d2ec721-4cb2-4735-9141-59acfbfded12',
 'error': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
           '@type': 'dspace:TransferError',
           'dspace:code': 'TRANSFER_ERROR_CODE',
           'dspace:consumerPid': '123',
           'dspace:providerPid': '123',
           'dspace:reason': ['Protocol Error. dspace:TransferRequestMessage is '
                             'not allowed here. Current state is '
                             'dspace:COMPLETED']}}
